In [1]:
import getpass
import os

# os.environ["OPENAI_API_KEY"] = getpass.getpass()

# from langchain_openai import ChatOpenAI

# model = ChatOpenAI(model="gpt-4o") # temperature 0.7 (default)

# from langchain_cohere import ChatCohere

# os.environ["COHERE_API_KEY"] = getpass.getpass("Enter your Cohere API Key: ")

# model = ChatCohere(model="command-r-plus-08-2024", temperature=0.7)

# from langchain_groq import ChatGroq

# model = ChatGroq(
#     model="mixtral-8x7b-32768",
#     temperature=0.7,
#     max_tokens=None,
#     timeout=None,
#     groq_api_key="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",
#     # max_retries=2,
#     # other params...
# )

from langchain_together import ChatTogether

model = ChatTogether(
    model="google/gemma-2-27b-it",
    temperature=0.7,
    max_tokens=None,
    timeout=None,
    api_key="5acc40de073869c904bc7929c58cd6b984435dfc6ff4655756f43eb256bb64db"
    # max_retries=2,
    # other params...
)

In [3]:
from langchain_core.output_parsers import StrOutputParser
from prompt import sundanese_overgeneration_prompt_gemma, topics
from tenacity import retry

chain = sundanese_overgeneration_prompt_gemma | model | StrOutputParser()

@retry
def generate_25_examples(topic):
    return chain.batch([{"topic":topic}] * 3)

In [8]:
len(responses)

252

In [9]:
from tqdm.auto import tqdm
# from langchain_core.tracers.context import tracing_v2_enabled
import os

# os.environ['LANGCHAIN_TRACING_V2'] = 'true'
# os.environ['LANGCHAIN_API_KEY'] = "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"


# responses = []
# topic_pointer = 0
# i=0

# with tracing_v2_enabled(project_name="gemma2-27B-sundanese-overgeneration"):
while len(responses) < 400:
    print(i)
    res = generate_25_examples(topics[topic_pointer])
    responses.extend(res)
    topic_pointer = (topic_pointer+1)%len(topics)
    i+=1

84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133


In [10]:
import re
def parse_generated_stories(res):
    # Split the input based on double new lines to separate each story
    res = re.sub(r'\d+\.\s*', '', res)
    res = res.split('\n')
    stories = []
    premise, correct_ending, incorrect_ending = None, None, None
    for line in res:
        if 'story premise' in line.lower():
            premise = line.split(':')[-1].strip()
        elif 'incorrect ending' in line.lower():
            incorrect_ending = line.split(":")[-1].strip()
            stories.append({"premise": premise, "correct_ending": correct_ending, "incorrect_ending": incorrect_ending})
            premise, correct_ending, incorrect_ending = None, None, None
        elif 'correct ending' in line.lower():
            correct_ending = line.split(':')[-1].strip()
        else:
            continue

    return stories

In [11]:
responses_formatted = [parse_generated_stories(res) for res in responses]
responses_final = [res for subset in responses_formatted for res in subset]

In [13]:
import pickle as pkl
import pandas as pd 

pkl.dump(responses_final, open("generated_stories.pkl", 'wb'))
pd.DataFrame(responses_final).to_csv("train_gemma2.csv", index=False)

In [12]:
len(responses_final)

1248